In [1]:
!pip install cmapy
!pip install nlpaug

  Preparing metadata (setup.py) ... done
  Created wheel for cmapy: filename=cmapy-0.6.6-py3-none-any.whl size=3940 sha256=32588bb5092627fb380e502bc2e4c746eef5b57d114b63736e225e4216203c35
  Stored in directory: /root/.cache/pip/wheels/f5/0c/06/46d29bf8036d0f70e7b0fdbdcd38747c0a2d0595d1a69a7885
Successfully built cmapy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 3.9 MB/s eta 0:00:00


## 필요 라이브러리

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
from scipy.io import wavfile
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from skimage.transform import resize
from torchvision import transforms
import torchaudio.transforms as ta_transforms
import math
import torchaudio
import time
import cmapy
import random
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['KMP_DUPLICATE_LIB_OK']

'True'

## 경로 설정

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/aiffel/AIFFELTHON

PATH = '/content/drive/MyDrive/aiffel/AIFFELTHON/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data'

Mounted at /content/drive
/content/drive/MyDrive/aiffel/AIFFELTHON


## 훈련, 테스트 데이터셋 분리

In [4]:
# txt파일 불러오기
file_list = os.listdir(PATH)
txt_list = [os.path.join(PATH, file) for file in file_list if file.endswith(".txt")]

# 환자 아이디를 훈련 데이터셋과 테스트 데이터셋으로 나눔
train_patient_txt, test_patient_txt = train_test_split(txt_list, test_size=0.9, random_state=42)

# # 결과 출력
# print("Train Patient IDs:", train_patient_txt[:3])
# print("Test Patient IDs:", test_patient_txt[:3])

## Customdataset

In [5]:
from customdataset import *

In [6]:
SAMPLE_RATE = 12000
HOP_LENGTH = 100
N_MELS = 128
N_FFT = 1024
WIN_LENGTH = 800

In [7]:
# 필터의 종류, 125Hz를 중간 주파수로 지정(심음 : 0~250Hz)
# 필터 타입, 주파수, 샘플레이트, Q
filters = [
    (Biquad.LOWPASS, 400, SAMPLE_RATE, 1.0),
    (Biquad.HIGHPASS, 25, SAMPLE_RATE, 1.0),
    (Biquad.BANDPASS, 125, SAMPLE_RATE, 1.0),
    (Biquad.NOTCH, 125, SAMPLE_RATE, 1.0),
    (Biquad.PEAK, 125, SAMPLE_RATE, 1.0, 1.0),
    (Biquad.LOWSHELF, 200, SAMPLE_RATE, 1.0, 1.0),
    (Biquad.HIGHSHELF, 125, SAMPLE_RATE, 1.0, 1.0)
]

In [8]:
dataset = CustomDataset(PATH, train_patient_txt,
                        sample_rate=SAMPLE_RATE,
                        hop_length=HOP_LENGTH,
                        n_mels=N_MELS,
                        n_fft=N_FFT,
                        win_length=WIN_LENGTH,
                        filter_params=[filters[0], filters[1], filters[5]],
                        padding_type=2,
                        multi_channels=False, clipping=True)

In [9]:
len(dataset.delete_list)
dataset.delete_list
print(len(dataset.y),len(dataset.x))
dataset.x.size(0)

1085 1085


1085

# **결과 시각화**

In [10]:
from matplotlib.patches import Rectangle

for i, d in enumerate(dataset):
    if i == 20: # 20개만 출력
        break
    x, y = d

    plt.figure(figsize=(10,8))
    fig, ax = plt.subplots()
    # 스펙트로그램 시각화
    if x.shape[-1] == 3:
        x = np.transpose(np.array(x), (1, 2, 0))
        librosa.display.specshow(cv2.cvtColor(np.array(x), cv2.COLOR_BGR2RGB),
                                 sr=SAMPLE_RATE,
                                 hop_length=HOP_LENGTH,
                                 n_fft=N_FFT,
                                 win_length=WIN_LENGTH,
                                 cmap='magma')
    else:
        librosa.display.specshow(np.array(x[0]),
                                 sr=SAMPLE_RATE,
                                 hop_length=HOP_LENGTH,
                                 n_fft=N_FFT,
                                 win_length=WIN_LENGTH,
                                 cmap='magma')

    # 레이블 시각화
    for label in y:
        start, _, end, height, class_id= label
        start *= 300    # 300 = output 이미지 가로 크기
        end *= 300      # 300 = output 이미지 가로 크기
        height *= 300   # 300 = output 이미지 세로 크기
        color = 'red' if class_id == 1 else 'blue'
        ax.add_patch(Rectangle((start, 0), (end - start), height,
                               edgecolor = color,
                               facecolor = 'white',
                               fill=True,
                               alpha=0.5,
                               lw=3))

    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.title(f"Spectrogram{i + 1} (dB) | clipping row={height/300}")

Output hidden; open in https://colab.research.google.com to view.